In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np

# Define your spatiotemporal transformer model
class SpatiotemporalTransformer(nn.Module):
    def __init__(self, num_frames=1, num_channels=1, hidden_dim=1, # ask about this, maybe not necessary
                 num_heads=1, num_layers=1):
#         super(SpatiotemporalTransformer, self).__init__()

#         # Spatial Transformer
#         self.spatial_transformer = nn.TransformerEncoderLayer(d_model=num_channels, nhead=num_heads)

#         # Temporal Transformer
#         self.temporal_transformer = nn.TransformerEncoderLayer(d_model=num_channels, nhead=num_heads)

#         # Positional encodings for spatial and temporal dimensions
#         self.spatial_pos_enc = nn.Parameter(torch.zeros(num_frames, num_channels))
#         self.temporal_pos_enc = nn.Parameter(torch.zeros(num_frames, num_channels))

#         # Fully connected layer for pixel prediction
#         self.fc = nn.Linear(num_channels, 3)  # Assuming RGB images

    def forward(self, x): # callable in nn.Module source code, runs whenever class is called with input
        
#         batch_size, num_frames, num_channels, height, width = x.size()

#         # Reshape the input to be compatible with the transformer
#         x = x.view(batch_size * num_frames, num_channels, height, width)

#         # Apply spatial transformation
#         x = x.permute(2, 3, 0, 1)  # (height, width, batch_size*num_frames, num_channels)
#         spatial_pos_enc = self.spatial_pos_enc.unsqueeze(1).repeat(1, height * width, 1)  # (num_frames, height*width, num_channels)
#         x = x + spatial_pos_enc
#         x = x.view(height * width, batch_size * num_frames, num_channels)  # (height*width, batch_size*num_frames, num_channels)
#         x = self.spatial_transformer(x)
#         x = x.view(height, width, batch_size, num_frames, num_channels)
#         x = x.permute(2, 3, 4, 0, 1)  # (batch_size, num_frames, num_channels, height, width)

#         # Apply temporal transformation
#         x = x.view(batch_size, num_frames, -1)  # (batch_size, num_frames, num_channels*height*width)
#         temporal_pos_enc = self.temporal_pos_enc.unsqueeze(1).repeat(1, num_frames, 1)  # (num_frames, num_frames, num_channels)
#         x = x + temporal_pos_enc
#         x = x.permute(1, 0, 2)  # (num_frames, batch_size, num_channels*height*width)
#         x = self.temporal_transformer(x)
#         x = x.permute(1, 0, 2)  # (batch_size, num_frames, num_channels*height*width)

#         # Pixel prediction
#         x = x.view(batch_size * num_frames, -1)  # (batch_size*num_frames, num_channels*height*width)
#         x = self.fc(x)
#         x = x.view(batch_size, num_frames, 3, height, width)  # (batch_size, num_frames, 3, height, width)

        return x

class Vegetation(Dataset): # inherited
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file) # replace with our file-reading
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


# Set up your dataset and data loaders
train_dataset = Vegetation(Dataset, targets)  # Replace with your training dataset class
test_dataset = Vegetation(Dataset, targets)    # Replace with your test dataset class

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1)

# Instantiate your spatiotemporal transformer model
model = SpatiotemporalTransformer()

# Define your loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the device (CPU or GPU) for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train() # forward()?
    train_loss = 0.0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)
        train_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)

    # Print training loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss}")

    # Model evaluation
    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Calculate loss
            loss = criterion(outputs, targets)
            test_loss += loss.item()

    test_loss /= len(test_loader)

    # Print test loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_loss}")

# Save the trained model
torch.save(model.state_dict(), "spatiotemporal_transformer.pt")


TypeError: object of type 'type' has no len()